In [1]:
using Pkg
Pkg.add("Distributions")
using JuMP, Gurobi, LinearAlgebra,Combinatorics, DelimitedFiles, Statistics, Distributions
# Abrir un solo ambiente de Gurobi, Jupyter se marea si se crean demasiados modelos durante en una misma celda.
const GUROBI_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09


    Updating registry at `C:\Users\Personal\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Personal\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Personal\.julia\environments\v1.7\Manifest.toml`


Gurobi.Env(Ptr{Nothing} @0x0000000042c76250, false, 0)

El problema que queremos resolver es:
\begin{equation}
    \overline{\mathcal{P}}:\quad \max\{g^{t}z : z\in Q, z\leq d\},
\end{equation}
donde $Q := \text{conv}(A)$, para $A$ una matriz con $m$ filas y $n$ columnas

## Problema 1

Demuestre que el problema $\overline{\mathcal{P}}$ es equivalente a
\begin{equation}
    \mathcal{P}:\quad \max\{(g^{t}A)x : \sum_{j=1}^{n}x_{j} = 1,\ Ax\leq d,\ x\geq0\}.
\end{equation}

**Demostración:**  Notemos que
\begin{equation}
    \left.\begin{array}{c}x\geq0\\\displaystyle\sum_{j=1}^{n}x_{j}=1\end{array}\right\}
    \iff
    \left\{\begin{array}{c}0\leq x\leq 1\\\displaystyle\sum_{j=1}^{n}x_{j}=1\end{array}\right.
\end{equation}

Además como $Q:=\text{conv}(A)$, lo escribimos como
\begin{equation}
    Q = \{y\in\mathbb{R}^{m} : Ax = y,\ 0\leq x\leq 1,\ \sum_{j=1}^{n}x_{j}=1\}.
\end{equation}

Por lo tanto, es evidente que reordenando el conjunto de la función a optimizar podemos ver la equivalencia entre los problemas deseados, en efecto:
\begin{eqnarray*}
    \overline{\mathcal{P}} 
    &=&    \max\{g^{t}z : z\in Q,\ z\leq d\}                                         \\
    &=&    \max\{g^{t}z : Ax = z,\ 0\leq x\leq 1,\ \sum_{j=1}^{n}x_{j}=1,\ z\leq d\} \\
    &=&    \max\{g^{t}Ax :0\leq x\leq 1,\ \sum_{j=1}^{n}x_{j}=1,\ Ax\leq d\}         \\
    &=&    \max\{g^{t}Ax :0\leq x,\ \sum_{j=1}^{n}x_{j}=1,\ Ax\leq d\}         \\
    &=& \mathcal{P}. \qquad \blacksquare
\end{eqnarray*}

## Problema 2
Usando *Julia/JuMP/GUROBI* programe el problema $\mathcal{P}$. Para ello, debe implementarlo en la función *modTI(A,d,g)* del notebook adjunto. Recibe como input una matriz $A$ de $n$ filas y $m$ columnas, junto con vectores $d,g$ de $n$ filas. Debe retornar e lmodelo planteado.

In [2]:
#===========================================================
Formato de objetos de un problema de optimizacion
===========================================================#
#     @variable(modelo, variables_con_sus_dominios, )
#     @objective(modelo, Min, expresion_de_la_funcion_objetivo)
#     @constraints(modelo, begin
#         primeraRestriccion ...
#     end)
function modTI(A,d,g,verboso=false)
    #===========================================================
    Inputs       =  A es una matriz de n x m, d y g son vectores 
                    de tamaño n.
    Outputs      =  Modelo que maximiza la funcion.
    Descripcion  =  Problema \mathcal{P}.
    ===========================================================#
    modelo = Model(optimizer_with_attributes(()->
            Gurobi.Optimizer(GUROBI_ENV),"Presolve"=>0, 
            "Cuts"=> 0, "Heuristics"=> 0, "Threads"=>1))
    if !verboso
        set_silent(modelo)
    end
    #===========================================================
    Variables
    ===========================================================#    
    @variable(modelo, 0 <= x[1:size(A)[2]])
    #===========================================================
    Restricciones
    ===========================================================#
    # Restriccion que indica que la suma de ponderadores es 1
    @constraint(modelo, sum(x[i] for i in 1:size(A)[2]) == 1)
    # Restriccion que indica que Ax <= d
    for i in 1:size(d)[1]
        @constraint(modelo, dot(A[i,:], x) <= d[i])
    end
    #===========================================================
    Funcion objetivo
    ===========================================================#
    @objective(modelo, Max, dot(g,A*x))
    return modelo
end

modTI (generic function with 2 methods)

## Problema 3
En el archivo *instancias.zip* se le entregan 11 matrices $A_{i}$ distintas, cada cual tiene $i$ filas y $2i$ columnas, con $i=5,\dots,15$, y sus entradas son solo $-1,0$ o $1$.

Para cada una de estas matrices $A_{i}$, defina y calcule (1) el vector $d_{A_{i}}\in\mathbb{R}^{i}$ tal que: $d_{A_{i}}(j)$ es el promedio de la $j$-ésima columna par de $A_{i}$, y (2) el vector $g_{A_{i}}\in\mathbb{R}^{i}$ tal que $g_{A_{i}}(j)$ es el pronmedio de la $j$-ésima columna impar de $A_{i}$. Para ello, debe completar el código respectivo en las funciones auxiliares *getd(A)* y *getg(A)* para obtener los vectores *d* y *g*, respectivamente, asumiendo que la entrada $A$ tiene $i$ filas y $2i$ colunas.

Una vez definida las funciones *getd* y *getg*, ejecute la función *getsol( )* para obtener el valor de $\mathcal{P}$ con las matrices $A$ de las instancias dadas, junto a los vectores $d_{A}$ y $g_{A}$.

In [3]:
function getd(A)
    #===========================================================
    Inputs       =  A es una matriz de i x 2i, d y g son 
                    vectores de tamaño n.
    Output       = Retorna un vector de tamaño i
    Descriopcion = Dada una matriz A de i filas y 2i columnas, 
                   calcula d como se especifica en el enunciado
    ===========================================================#
    # Obtenemos el tamaño de la matriz
    n = size(A)[1] 
    # Definimos un vector d
    d = zeros(n)
    for j in 1:n
        d[j] = mean(A[:,2*j])
    end
    output = d
    return output
end

getd (generic function with 1 method)

In [10]:
function getg(A)
    #===========================================================
    Inputs       =  A es una matriz de i x 2i, d y g son 
                    vectores de tamaño n.
    Output       = Retorna un vector de tamaño i
    Descriopcion = Dada una matriz A de i filas y 2i columnas, 
                   calcula g como se especifica en el enunciado
    ===========================================================#
    # Obtenemos el tamaño de la matriz
    n = size(A)[1] 
    # Definimos un vector d
    g = zeros(n)
    for j in 1:n
        g[j] = mean(A[:,2*j-1])
    end
    output = g
    return output
end

getg (generic function with 1 method)

In [5]:
# Test de los metodos getd() y getg()
A = [1 2 3 4; 
    10 20 30 40]
show([getd(A)  getg(A)])

[11.0 5.5; 22.0 16.5]

In [6]:
#Funcion para cargar instancias A, calcular d y g, obtener el modelo y su valor en el optimo.
function getsol()
    for i in 1:11
        println("INSTANCIA "* string(i))
        nm = "instancia_" * string(i) * ".txt"
        A=readdlm(nm, ' ', Int, '\n')
        #print(A)
        d = getd(A)
        #print(d)
        g = getg(A)
        #print(g)
        #println(A)
        modelo = modTI(A,d,g);
        optimize!(modelo);
        if termination_status(modelo) == MOI.OPTIMAL
            print("El valor objetivo es: ")
            println(objective_value(modelo))
        end
    end
    return
end

getsol (generic function with 1 method)

In [7]:
#Ejecute get_sol()
getsol()

INSTANCIA 1
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
El valor objetivo es: -0.06000000000000004
INSTANCIA 2
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
El valor objetivo es: -0.05555555555555569
INSTANCIA 3
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
El valor objetivo es: 0.14285714285714285
INSTANCIA 4
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
El valor objetivo es: 1.375
INSTANCIA 5
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
El valor objetivo es: 1.1049382716049383
INSTANCIA 6
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Se

## Problema 4
Suponga que es necesario agregar $p$ columnas a la matriz $A$, dadas por una matriz $B$. Usando la solución óptima $\overline{x}$ del problema original, constuya una solución factible del nuevo problema
\begin{equation}
    \mathcal{P}^{+}: \quad \max\{ (g^{t}A')u : \sum_{j=1}^{n+p} u_{j} = 1,\ A'u\leq d, u\geq 0\},
\end{equation}
donde $A'$ es la matriz que tiene tanto las columnas de $A$ como las de $B$, osea $A'=[A|B]$.

**Solución:** Sea $\displaystyle x \in \arg\max\{ q^{t}Ax : \sum_{j=1}^{n} x_{j} = 1,\ Ax\leq d, x\geq 0\}$, notemos que
\begin{equation}
    u :=
    \begin{pmatrix}
        x\\
        0_{p}
    \end{pmatrix}
\end{equation}
es tal que
\begin{eqnarray*}
    \sum_{j=1}^{n} x_{j} + \sum_{j=1}^{p} 0 = 1,\qquad
    [A|B]u = Ax + B0_{p} = Ax \leq d,\qquad
    u\geq0,
\end{eqnarray*}
entonces $u$ es un punto factible de $\mathcal{P}^{+}$.

**Demostración alternativa:** Sea $x$ solución óptima de $\mathcal{P}$, por lo tanto
\begin{eqnarray*}
    \sum_{j=1}^{n} x_{j} &=& 1\\
    x_{j} &\geq& 0,
\end{eqnarray*}
notemos que para el caso base en que $p=1$ sigue que podemos extender los ponderadores convexos mediante
\begin{eqnarray*}
    u_{j}^{1} = 
    \begin{cases}
        \dfrac{x_{j}}{u_{n+1}^{1}}                        & j < n,\\
        \displaystyle 1 - \sum_{k=1}^{n}u_{j}^{1}         & j = n+1.
    \end{cases}
\end{eqnarray*}

Luego, de forma recursiva tenemos que la $p$-ésima extensión depende de la $(p-1)$-ésima extensión, en efecto, 
\begin{eqnarray*}
    u_{j}^{p} =
    \begin{cases}
        \dfrac{u_{j}^{p-1}}{u_{n+p}^{p}}                   & j < n+p,\\
        \displaystyle 1 - u_{j}^{p}                        & j = n+p.
    \end{cases}
\end{eqnarray*}

Es evidente que para $p\geq1$, la $p$-ésima extensión convexa cumple que
\begin{eqnarray*}
    (\forall j=1,\dots,n+p),\ u_{j}^{p} \geq0 \quad\text{y}\quad \sum_{j=1}^{n+p} u_{j}^{p} = 1.
\end{eqnarray*}

Finalmente veamos que
\begin{equation}
    Ax \leq d \Longrightarrow [A,B]u^{p} \leq d.
\end{equation}

En efecto, razonando de igual manera, escribimos $A = (A_{j})_{j=1}^{n}$ y $B = (B_{j})_{j=1}^{p}
$ las colecciones de vectores columnas de $A$ y $B$ respectivamente, tenemos que para $p=1$ consideramos la primera extensión $u^{1} = (u_{j}^{1})_{j=1}^{n+1}$, entonces
\begin{eqnarray*}
    B_{1}u_{n+1}^{1} + \sum_{j=1}^{n} A_{j}u_{j}^{1}
    &=& B_{1}u_{n+1}^{1} + \frac{1-u_{n+1}^{1}}{1-u_{n+1}^{1}} \sum_{j=1}^{n} A_{j}u_{j}^{1}\\
    &=& B_{1}u_{n+1}^{1} + (1-u_{n+1}^{1})\sum_{j=1}^{n} \frac{u_{j}^{1}}{1-u_{n+1}^{1}} A_{j},
\end{eqnarray*}
donde tenemos que $\left(\frac{u_{j}^{1}}{1-u_{n+1}^{1}}\right)_{j=1}^{n}$ es factible para $\mathcal{P}$, y por lo tanto
\begin{eqnarray*}
    \sum_{j=1}^{n} \frac{u_{j}^{1}}{1-u_{n+1}^{1}} A_{j} \leq d,
\end{eqnarray*}
por lo tanto (*)
\begin{eqnarray*}
    B_{1}u_{n+1}^{1} + \sum_{j=1}^{n} A_{j}u_{j}^{1} 
    &=&    B_{1}u_{n+1}^{1} + (1-u_{n+1}^{1})\sum_{j=1}^{n} \frac{u_{j}^{1}}{1-u_{n+1}^{1}} A_{j}\\
    &\leq& u_{n+1}^{1}d + (1-u_{n+1}^{1})d \\
    &=&    (u_{n+1}^{1} + 1-u_{n+1}^{1}) d \\ 
    &=&    d 
\end{eqnarray*}

Mientras que para $p\geq2$ consideramos la $p$-ésima extensión $u^{p} = (u_{j}^{p})_{j=1}^{n+p-1}$, entonces
\begin{eqnarray*}
    \sum_{j=1}^{p}B_{j}u_{n+j}^{p} + \sum_{j=1}^{n} A_{j}u_{j}^{p}
    &=& B_{p}u_{n+p}^{p} + \frac{1-u_{n+p}^{p}}{1-u_{n+p}^{p}} \left(\sum_{j=1}^{n} A_{j}u_{j}^{p} + \sum_{j=1}^{p-1} B_{j}u_{n+j}^{p}\right)\\
    &\leq& u_{n+p}^{p}d + (1-u_{n+p}^{p})d\\
    &=&    (u_{n+1}^{p}+1-u_{n+p}^{p}) d\\
    &=&    d
\end{eqnarray*}    

Por lo tanto, hemos escrito $u\in\text{fact}(\mathcal{P}^{+})$ en términos de $x\in\text{fact}(\mathcal{P})$ mediante recursividad. $\qquad \blacksquare$

## Problema 5
Encuentre el dual de $\mathcal{P}^{+}$ y úselo para proponer un test para decidir si la solución factible propuesta en la parte anterior es óptima.

**Preliminar:** Recordemos que la relación entre el problema primal $\mathcal{P}$ y dual $\mathcal{D}$ viene dada por
\begin{equation}
    (\mathcal{P})\ \max\{c^{t}x : Ax\leq b\} \qquad (\mathcal{D})\ \min\{b^{T}y : A^{t}y=c,y\geq0\}.   
\end{equation}

**Solución:** Tenemos el *problema primal*
\begin{equation}
    \mathcal{P}^{+}: \quad \max\{ (g^{t}A')u : \sum_{j=1}^{n+p} u_{j} = 1,\ A'u\leq d, u\geq 0\}.
\end{equation}

Es evidente que podemos reformular $\mathcal{P}^{+}$ puesto que
\begin{equation}
    \sum_{j=1}^{n+p} u_{j} = 1,\ A'u\leq d, u\geq 0 
    \underset{\begin{subarray}{c}\text{Escribiendo}\\ \text{bloques}\end{subarray}}{\iff}
    \begin{pmatrix}
        A'&\Theta_{n+p}&\Theta_{n+p}\\
        \Theta_{n+p}&-I_{n+p}&\Theta_{n+p}\\
        \theta_{n+p}^{t}&\theta_{n+p}^{t}&1_{n+p}^{t}\\
        \theta_{n+p}^{t}&\theta_{n+p}^{t}&-1_{n+p}^{t}
    \end{pmatrix}
    \begin{pmatrix}
        u\\
        u\\
        u
    \end{pmatrix}
    \leq
    \begin{pmatrix}
        d\\
        \theta_{n+p}\\
        1\\
        1
    \end{pmatrix}
    \underset{\begin{subarray}{c}\text{Representación}\\\text{conveniente}\end{subarray}}{\iff}
    \begin{pmatrix}
        A'\\
        -I_{n+p}\\
        1_{n+p}^{t}\\
        -1_{n+p}^{t}
    \end{pmatrix}
    u
    \leq
    \begin{pmatrix}
        d\\
        0_{n+p}\\
        1\\
        1
    \end{pmatrix}
\end{equation}
donde $\theta_{n+p} = (0_{i})_{i=1}^{n+p}$, $\Theta_{n+p} = (0_{ij})_{i,j=1}^{n+p}$, $1_{n+p} = (1_{i})_{i=1}^{n+p}$ e $I_{n+p} = (1_{ij})_{i,j=1}^{n+p}$, donde hemos usado que
\begin{equation}
    \sum_{j=1}^{n+p} u_{j} = 1 \iff 
    \sum_{j=1}^{n+p} u_{j} \leq 1,\ \sum_{j=1}^{n+p} u_{j} \leq -1.
\end{equation}

Sean 
\begin{eqnarray*}
    \gamma^{t} = q^{t}A'
    ,\qquad 
    \alpha =
    \begin{pmatrix}
        A'\\
        -I_{n+p}\\
        1_{n+p}^{t}\\
        -1_{n+p}^{t}
    \end{pmatrix}
    ,\qquad
    \beta =
    \begin{pmatrix}
        d\\
        0_{n+p}\\
        1\\
        1
    \end{pmatrix},
\end{eqnarray*}
entonces
\begin{equation}
    \mathcal{P}^{+}: \quad \max\{\gamma^{t}u : \alpha u \leq \beta\}.
\end{equation}

Finalmente, se concluye que 
\begin{equation}
    \mathcal{D}^{+}: \quad \min\{\beta^{t}v : \alpha^{t}v = \gamma,\ v\geq0\}.
\end{equation}

Para saber cuando una solución es óptima nos basta con estudiar la condición de *holgura complementaria*, esto es que $\overline{u}$ y $\overline{v}$ son óptimos de $\mathcal{P}^{+}$ y $\mathcal{D}^{+}$ respectivamente, ssi
\begin{equation}
    (\forall i =1,\dots,m+(n+p) + 2),\qquad \overline{v}_{i} (\alpha_{i} \overline{u} - \beta_{i}) = 0,
\end{equation}
donde $\alpha_{i}$ corresponde a la fila $i$-ésima de la matriz $\alpha$. 

Veamos ahora un test para determinar cuando la solución factible $u = \begin{pmatrix}x \\ 0_{p}\end{pmatrix}$  es óptima, usando la condición de *holgura complementaria* sigue que para $\overline{v}$ solución óptima de $\mathcal{D}^{+}$:
\begin{equation}
    \begin{array}{lrcl}
        (\forall i=1,\dots,m),          & \overline{v}_{i}(A_{i}x + B_{i}0 - d_{i})                               &=& 0\\
        (\forall i=1,\dots,n),          & -\overline{v}_{m+i}x_{i}                                                &=& 0\\
        (\forall i=1,\dots,p),          & \overline{v}_{m+n+i} \cdot 0                                            &=& 0\\
                                        & \overline{v}_{m+n+p+1}\left(\displaystyle\sum_{j=1}^{m}x_{j} - 1\right) &=& 0\\
                                        & \overline{v}_{m+n+p+2}\left(-\displaystyle\sum_{j=1}^{m}x_{j} - 1\right)&=& 0,
    \end{array}
\end{equation}
lo cual, al ser simplificado lo podemos expresar como
\begin{equation}
    \begin{array}{lrcl}
        (\forall i=1,\dots,m),          & \overline{v}_{i}(A_{i}x - d_{i})                                        &=& 0\\
        (\forall i=1,\dots,n),          & \overline{v}_{m+i}x_{i}                                                 &=& 0\\
                                        & \overline{v}_{m+n+p+1}\left(\displaystyle\sum_{j=1}^{m}x_{j} - 1\right) &=& 0\\
                                        & \overline{v}_{m+n+p+2}\left(-\displaystyle\sum_{j=1}^{m}x_{j} - 1\right)&=& 0,
    \end{array}
 \end{equation}
 por lo tanto, basta con obtener un punto óptimo $\overline{v}$ del problema $\mathcal{D}^{+}$ y luego *resolver el sistema de ecuaciones lineales* antes presentado, más aún, el punto factible obtenido a partir de $x$ resultará ser óptimo si resulta pertenecer al conjunto solución del sistema de ecuaciones lineales planteado antes.

## Problema 6

Aplique el test a las instancias del **problema 3** para matrices $B(i)$ cuyas columnas corresponden a las indicatrices de todos los subconjuntos de $\{1,\dots,i\}$ con $i-2$ elementos. Es decir, ejecute el programa diseñado en el **problema 2** con las matrices $A' = [A|B]$ tal que $B$ es una matriz que tiene $\binom{n}{n-2}$ columnas, y cada una de ellas corresponde a la indicatriz de un subconjunto de tamaño $i-2$ de $[i]$ ($i$ es la cantidad de filas de $A$). Por ejemplo, si $i=5$, entonces $\{1,3,5\}$ es un subconjunto de tamaño $i-2$ y entonces su columna respectiva en $B$ es $(1,0,1,0,1)^{t}$. Para realizar esto, complete la función *getsolB( )* siguiendo las instrucciones ahí especificadas.

En la sección $\texttt{CDG1}$ debe:
 - Calcular la matriz B como se define en el enunciado
 - Definir $A' = [A|B]$.
 - Calcular $d_{A'}$ y $g_{A'}$.
 - Obtener el modelo para $A', d_{A'}, g_{A'}$

En la sección $\texttt{CDG2}$ debe realizar el testeo que definió en la $\textbf{P5}$. Si ya resolvió el primal con $\texttt{GUROBI}$, investigue como puede obtener el valor del problema dual, junto con el valor de las variables duales en el óptimo. 

Dentro de $\texttt{CDG2}$ debe realizar un $\textit{print}$ para cada instancia, indicando si se cumple el testeo realizando y justificando por qué.

**Matriz $B$:** Notemos que la matriz $B$ tiene sus filas indexadas por el conjunto $I$ de cardinal $i$, mientras que sus columnas están indexadas por todos los subconjuntos de $I$ de cardinal $i-2$, osea $B$ posee $\binom{i}{i-2}$ columnas. Notemos que cada columna posee $2$ ceros, de hecho, podemos entender la matriz $B$ como la matriz conformada por todos los vectores de $1$ y $0$ que poseen exactamente dos $0$s, osea
\begin{equation}
    B =
    \begin{pmatrix}
        0&\cdots&\cdots&1\\
        0&\cdots&\cdots&1\\
        1&\cdots&\cdots&1\\
        \vdots&\ddots&\ddots&\vdots\\
        1&\cdots&\cdots&0\\
        1&\cdots&\cdots&0\\
    \end{pmatrix} \in \mathcal{M}_{|I|\times\left|\binom{i}{i-2}\right|}(\{0,1\}).
\end{equation}
Para calcular la matriz $B$ usamos el paquete *combinatorics*.

In [8]:
#Funcion para cargar instancias A, calcular B, definir A' = [A|B] d_ y g, obtener el modelo y su valor en el optimo.
function getsolB()
    for i in 1:11
        println("INSTANCIA "* string(i))
        nm = "instancia_" * string(i) * ".txt"
        A=readdlm(nm, ' ', Int, '\n')
        
        #-----------------------------------------------------
        #    CDG1
        #-----------------------------------------------------
        
        optimize!(modelo);
        if termination_status(modelo) == MOI.OPTIMAL
            #------------------------------------------
            #     CDG2
            #------------------------------------------
        end
    
    end
    return
end

getsolB (generic function with 1 method)

In [9]:
#Ejecute getsolB

getsolB()

INSTANCIA 1


LoadError: UndefVarError: modelo not defined